# Training Word2Vec on Yelp reviews
Full dataset (8M) tested on `16 vCPUs, 128 GB RAM`, 1M sample tested on `8 vCPUs, 30 GB RAM`

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.  
https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/word2vec.ipynb


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [1]:
#Ensure gensim  and Cython are installed
# !pip list

In [2]:
# import modules & set up logging
import os
import pandas as pd
from pandarallel import pandarallel

import re
import nltk
import gensim

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 200)

In [3]:
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

Available CPUs: 16


#### Copy files to local FS from GCP bucket

In [4]:
!mkdir -p /home/jupyter/data/wordvec

In [5]:
# !gsutil -m cp -n 'gs://msca-bdp-data-open/wordvec/questions-words.txt' '/home/jupyter/data/wordvec/'

### Training Word2vec on Yelp Reviews

In [6]:
%%time

# path_read = 'https://storage.googleapis.com/msca-bdp-data-open/yelp/yelp_academic_dataset_review.json'
# yelp_df = pd.read_json(path_read, orient='records', lines=True, nrows=1000000)
# yelp_df = yelp_df[['text']]

path_read = 'https://storage.googleapis.com/msca-bdp-data-open/yelp/yelp_academic_dataset_review.parquet'
# yelp_df = pd.read_parquet(path_read, engine='pyarrow', columns=['text']).head(1000000)
yelp_df = pd.read_parquet(path_read, engine='pyarrow', columns=['text'])


print(f'Memory used by DF {yelp_df.memory_usage().sum()}')
print(f'Read rows: {yelp_df.shape[0]}, columns: {yelp_df.shape[1]}')

Memory used by DF 64169104
Read rows: 8021122, columns: 1
CPU times: user 21.5 s, sys: 15.1 s, total: 36.6 s
Wall time: 46.4 s


In [7]:
yelp_df.head(5)

,text
0,"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the ..."
1,I am actually horrified this place is still in business. My 3 year old son needed a haircut this past summer and the lure of the $7 kids cut signs got me in the door. We had to wait a few minutes ...
2,"I love Deagan's. I do. I really do. The atmosphere is cozy and festive. The shrimp tacos and house fries are my standbys. The fries are sometimes good and sometimes great, and the spicy dipping sa..."
3,"Dismal, lukewarm, defrosted-tasting ""TexMex"" glop;\n\nMumbly, unengaged waiter;\n\nClueless manager, who seeing us with barely nibbled entrees\non plates shoved forward for pickup, thanked us\nper..."
4,"Oh happy day, finally have a Canes near my casa. Yes just as others are griping about the Drive thru is packed just like most of the other canes in the area but I like to go sit down to enjoy my c..."


#### Tokenize text into Sentences

In [8]:
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [9]:
%%time

# Convert text into sentences

# yelp_df["sentences"] = yelp_df.apply(lambda x: nltk.tokenize.sent_tokenize(x['text']), axis=1)
yelp_df["sentences"] = yelp_df.parallel_apply(lambda x: nltk.tokenize.sent_tokenize(x['text']), axis=1)
yelp_df = yelp_df[["sentences"]]

# Create a dataframe with one sentence per row
yelp_df = yelp_df.explode("sentences", ignore_index=True)

print(f'Rows: {yelp_df.shape[0]}, Columns: {yelp_df.shape[1]}')

Rows: 64098542, Columns: 1
CPU times: user 54.1 s, sys: 24.9 s, total: 1min 18s
Wall time: 5min 56s


#### 

In [10]:
yelp_df.head(5)

,sentences
0,"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas."
1,"When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!"
2,"Tucked away near the gelateria and the garden, the Gallery is pretty much hidden from view."
3,"It's what real estate agents would call ""cozy"" or ""charming"" - basically any euphemism for small."
4,"That being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask?"


#### Clean-up text

In [11]:
%%time

stop_words = nltk.corpus.stopwords.words('english')

yelp_df['clean_sentences'] = yelp_df['sentences'].parallel_apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
yelp_df['clean_sentences'] = yelp_df['clean_sentences'].parallel_apply(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', x))

yelp_df = yelp_df[['clean_sentences']]

CPU times: user 1min 49s, sys: 1min 9s, total: 2min 59s
Wall time: 6min 11s


In [12]:
yelp_df.head(5)

,clean_sentences
0,"As someone worked many museums, I eager visit gallery recent trip Las Vegas."
1,"When I saw would showing infamous eggs House Faberge Virginia Museum Fine Arts VMFA, I knew I go"
2,"Tucked away near gelateria garden, Gallery pretty much hidden view."
3,Its real estate agents would call cozy charming basically euphemism small.
4,"That said, still see wonderful art gallery size, two s ask"


In [13]:
sentences = [row.split() for row in yelp_df['clean_sentences']]
sentences[:2]

[['As',
  'someone',
  'worked',
  'many',
  'museums,',
  'I',
  'eager',
  'visit',
  'gallery',
  'recent',
  'trip',
  'Las',
  'Vegas.'],
 ['When',
  'I',
  'saw',
  'would',
  'showing',
  'infamous',
  'eggs',
  'House',
  'Faberge',
  'Virginia',
  'Museum',
  'Fine',
  'Arts',
  'VMFA,',
  'I',
  'knew',
  'I',
  'go']]

In [14]:
del yelp_df

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

### size
`vector_size ` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [15]:
%%time

# default value of min_count=5
# default value of size=100
# default value of workers=3
# sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
workers = num_processors-1

model = gensim.models.Word2Vec(sentences, min_count=10, vector_size=100, compute_loss=True, sg=0, workers=workers)

CPU times: user 2h 29min 14s, sys: 1min 35s, total: 2h 30min 50s
Wall time: 50min 27s


The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Training Loss Computation

The parameter `compute_loss` can be used to toggle computation of loss while training the Word2Vec model. The computed loss is stored in the model attribute `running_training_loss` and can be retrieved using the function `get_latest_training_loss` as follows : 

In [16]:
%%time

# getting the training loss value
training_loss = model.get_latest_training_loss()
print(training_loss)

79790696.0
CPU times: user 198 µs, sys: 4 µs, total: 202 µs
Wall time: 145 µs


## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [17]:
directory_w2v = '/home/jupyter/data/wordvec/'

In [18]:
%%time

evaluation = model.wv.evaluate_word_analogies(directory_w2v + 'questions-words.txt')

CPU times: user 12min 12s, sys: 17min 22s, total: 29min 34s
Wall time: 1min 51s


In [19]:
evaluation[0]

0.31187331443192035

In [20]:
evaluation[1][7]

{'section': 'gram3-comparative',
 'correct': [('BAD', 'WORSE', 'BIG', 'BIGGER'),
  ('BAD', 'WORSE', 'BRIGHT', 'BRIGHTER'),
  ('BAD', 'WORSE', 'CHEAP', 'CHEAPER'),
  ('BAD', 'WORSE', 'EASY', 'EASIER'),
  ('BAD', 'WORSE', 'FAST', 'FASTER'),
  ('BAD', 'WORSE', 'GOOD', 'BETTER'),
  ('BAD', 'WORSE', 'HARD', 'HARDER'),
  ('BAD', 'WORSE', 'HIGH', 'HIGHER'),
  ('BAD', 'WORSE', 'LARGE', 'LARGER'),
  ('BAD', 'WORSE', 'LONG', 'LONGER'),
  ('BAD', 'WORSE', 'LOUD', 'LOUDER'),
  ('BAD', 'WORSE', 'LOW', 'LOWER'),
  ('BAD', 'WORSE', 'QUICK', 'QUICKER'),
  ('BAD', 'WORSE', 'SAFE', 'SAFER'),
  ('BAD', 'WORSE', 'SLOW', 'SLOWER'),
  ('BAD', 'WORSE', 'SMALL', 'SMALLER'),
  ('BAD', 'WORSE', 'SMART', 'SMARTER'),
  ('BAD', 'WORSE', 'STRONG', 'STRONGER'),
  ('BAD', 'WORSE', 'TALL', 'TALLER'),
  ('BAD', 'WORSE', 'TIGHT', 'TIGHTER'),
  ('BAD', 'WORSE', 'TOUGH', 'TOUGHER'),
  ('BAD', 'WORSE', 'WIDE', 'WIDER'),
  ('BIG', 'BIGGER', 'BRIGHT', 'BRIGHTER'),
  ('BIG', 'BIGGER', 'CHEAP', 'CHEAPER'),
  ('BIG', 'BIGGER', 

This `evaluate_word_analogies` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In newer release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [21]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep

In [22]:
model.wv.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

((0.5214319113422861, 4.243988254763822e-25),
 SpearmanrResult(correlation=0.5071095832675625, pvalue=1.2874591355880186e-23),
 3.6827195467422094)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [23]:
save_w2v_model = '/home/jupyter/data/wordvec/yelp_model/yelp.model'

model.save("yelp.model")  # save the model

In [24]:
!rm -r /home/jupyter/data/wordvec/yelp_model
!mkdir -p /home/jupyter/data/wordvec/yelp_model

In [25]:
!mv yelp.model* '/home/jupyter/data/wordvec/yelp_model/'

In [26]:
!gsutil -m cp '/home/jupyter/data/wordvec/yelp_model/*' 'gs://msca-bdp-data-open/wordvec/yelp_model/'
# !gsutil -m cp 'gs://msca-bdp-data-open/wordvec/yelp_model/*' '/home/jupyter/data/wordvec/yelp_model/' 

Copying file:///home/jupyter/data/wordvec/yelp_model/yelp.model.wv.vectors.npy [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/data/wordvec/yelp_model/yelp.model [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/data/wordvec/yelp_model/yelp.model.syn1neg.npy [Content-Type=application/octet-stream]...
/ [3/3 files][261.2 MiB/261.2 MiB] 100% Done                                    
Operation completed over 3 objects/261.2 MiB.                                    


In [27]:
new_model = gensim.models.Word2Vec.load(save_w2v_model)  # open the model

which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [28]:
if 'tea' in model.wv:
    print(model.wv['tea'].shape)
else:
    print('{0} is an out of dictionary word'.format('tea'))

(100,)


In [29]:
model.wv.most_similar(positive=['tea', 'coffee'], negative=['sugar'], topn=10)

[('teacoffee', 0.711517870426178),
 ('tea....', 0.6996362209320068),
 ('coffee,', 0.6959604024887085),
 ('cofee', 0.693589448928833),
 ('teas', 0.691807210445404),
 ('coffees', 0.6913444995880127),
 ('tea,', 0.6880320310592651),
 ('coffee.', 0.6837782859802246),
 ('coffe', 0.6798722147941589),
 ('tea..', 0.6728952527046204)]

In [30]:
print(model.wv.similarity('tea', 'coffee'))
print(model.wv.similarity('bread', 'butter'))

0.69982266
0.6391462


You can get the probability distribution for the center word given the context words as input:

In [31]:
print(model.predict_output_word(['bread', 'butter', 'toast']))

[('french', 0.3474941), ('sourdough', 0.2389522), ('raisin', 0.038162634), ('toasted', 0.03587312), ('French', 0.034209155), ('rye', 0.029932462), ('buttered', 0.023783017), ('butter', 0.019234119), ('pudding', 0.012823981), ('pudding,', 0.012651101)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [32]:
model.wv.__getitem__('bread')  # raw NumPy vector of a word

array([-1.3861886e+00, -3.3910768e+00,  2.9922774e+00, -1.5011976e-02,
        1.2577108e+00,  5.4005389e+00,  2.2296293e+00, -7.8657651e-01,
        4.6269779e+00, -1.3002990e+00, -4.2794850e-01,  3.6609678e+00,
        2.3330035e+00,  6.3360316e-01,  6.8322377e+00, -8.1748801e-01,
       -3.4931455e+00, -5.0601357e-01,  1.6484153e+00,  2.2959580e+00,
       -1.8920726e+00,  3.9411697e+00, -3.5330054e-01,  3.7190220e+00,
       -1.0366001e+00,  6.9146717e-01, -1.9829515e+00,  8.5843456e-01,
       -2.9597306e-01, -3.5340636e+00,  5.0987225e+00, -2.5290754e+00,
       -2.4001811e+00,  2.7242334e+00, -3.0010340e-01, -3.2647185e+00,
        1.9655538e+00, -4.5680785e+00,  1.7520018e-01,  5.7515442e-01,
       -6.7947811e-05,  3.3135030e+00,  1.1133091e+00, -2.8905514e-01,
       -2.3900666e+00, -3.4701898e+00, -2.3195388e+00, -2.0346200e+00,
        3.9802620e+00,  3.0390911e+00, -2.5262439e+00, -2.8347275e+00,
       -2.1344610e-01, -1.5069736e+00, -1.2601237e+00, -6.3722568e+00,
      

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).

In [33]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Tue, 16 November 2021 08:44:08'